In [8]:
!pip install sagemaker-experiments

  Using cached sagemaker_experiments-0.1.42-py3-none-any.whl (42 kB)

[notice] A new release of pip available: 22.3.1 -> 23.0.1
[notice] To update, run: pip install --upgrade pip


In [9]:
import sagemaker
import boto3
import pandas as pd
from datetime import datetime 

from sagemaker import image_uris 
from sagemaker.session import Session
from sagemaker.inputs import TrainingInput
from time import gmtime, strftime
import time
from smexperiments import experiment

region = boto3.Session().region_name

session = sagemaker.Session()
bucket = 'lawsnic-aiml-east2'
prefix = 'kaggle/customerChurn'

role = sagemaker.get_execution_role()

sm = boto3.Session().client(service_name='sagemaker',region_name=region)


# name of experiment
timestep = datetime.now()
timestep = timestep.strftime("%d-%m-%Y-%H-%M-%S")
experiment_name = timestep + "xgboost-cust-churn"

# create experiment
experiment.Experiment.create(
    experiment_name=experiment_name,
    description="Iterative model tuning - altering data engineering steps",
    sagemaker_boto_client=sm,
)

Experiment(sagemaker_boto_client=<botocore.client.SageMaker object at 0x7f8b46cfa590>,experiment_name='18-02-2023-16-20-58xgboost-cust-churn',description='Iterative model tuning - altering data engineering steps',tags=None,experiment_arn='arn:aws:sagemaker:us-east-2:791580863750:experiment/18-02-2023-16-20-58xgboost-cust-churn',response_metadata={'RequestId': '85bbc015-ef1d-47ef-99c3-f23b5c20e90a', 'HTTPStatusCode': 200, 'HTTPHeaders': {'x-amzn-requestid': '85bbc015-ef1d-47ef-99c3-f23b5c20e90a', 'content-type': 'application/x-amz-json-1.1', 'content-length': '109', 'date': 'Sat, 18 Feb 2023 16:20:58 GMT'}, 'RetryAttempts': 0})

In [10]:
#sm.get_paginator('list_candidates_for_auto_ml_job')
#sm.list_candidates_for_auto_ml_job(AutoMLJobName='xgb-custChurn-2023-2-18-11')

In [11]:
# cell 10
container = sagemaker.image_uris.retrieve(region=boto3.Session().region_name, framework='xgboost', version='latest')

INFO:sagemaker.image_uris:Ignoring unnecessary instance type: None.


In [17]:
# cell 11
#validation_location = 's3://lawsnic-aiml-east2/kaggle/customerChurn/features/partial/test/CustomerChurnDW-2023-01-06T17-24-14/part-00000-3986a971-e8a5-4844-bfc8-e72e78f341b4-c000.csv'
#training_location = 's3://lawsnic-aiml-east2/kaggle/customerChurn/features/partial/train/CustomerChurnDW-2023-01-06T17-24-14/part-00000-0338ba51-05d0-4737-bd40-d52e24739eb6-c000.csv'

training_location = 's3://lawsnic-aiml-east2/kaggle/customerChurn/features/partial/train/CustomerChurnDW-2023-01-06T14-14-13/part-00000-642a0746-49b9-4fda-ad2d-98afe1db11ec-c000.csv'
validation_location = 's3://lawsnic-aiml-east2/kaggle/customerChurn/features/partial/test/CustomerChurnDW-2023-01-06T14-14-13/part-00000-ccd6e4dd-898c-4fc4-a63a-85d1cfcfc4dc-c000.csv'

In [28]:
df = pd.read_csv(training_location)
df["SeniorCitizen"] = df["SeniorCitizen"].astype(int)
df["Churn"] = df["Churn"].astype(int)
df["Partner"] = df["Partner"].astype(int)
df["Dependents"] = df["Dependents"].astype(int)
df["PhoneService"] = df["PhoneService"].astype(int)
df["PaperlessBilling"] = df["PaperlessBilling"].astype(int)
df["isFemale"] = df["isFemale"].astype(int) 
df.to_csv('s3://lawsnic-aiml-east2/kaggle/customerChurn/features/partial/train_with_header.csv',index=False)
df.to_csv('s3://lawsnic-aiml-east2/kaggle/customerChurn/features/partial/train.csv',header=None,index=False)

print('train data has {} features'.format(len(df.columns)))

df = pd.read_csv(validation_location)
df["SeniorCitizen"] = df["SeniorCitizen"].astype(int)
df["Churn"] = df["Churn"].astype(int)
df["Partner"] = df["Partner"].astype(int)
df["Dependents"] = df["Dependents"].astype(int)
df["PhoneService"] = df["PhoneService"].astype(int)
df["PaperlessBilling"] = df["PaperlessBilling"].astype(int)
df["isFemale"] = df["isFemale"].astype(int)
df.to_csv('s3://lawsnic-aiml-east2/kaggle/customerChurn/features/partial/validate_with_target_and_header.csv',index=False)
df.to_csv('s3://lawsnic-aiml-east2/kaggle/customerChurn/features/partial/validate_with_target.csv',header=None,index=False)

print('validation data has {} features before dropping target'.format(len(df.columns)))

test_data2 =df.drop(["Churn"], axis=1) 
test_data2.to_csv('s3://lawsnic-aiml-east2/kaggle/customerChurn/features/partial/validate.csv',header=None,index=False)

df = 0
test_data2=0

s3_uri_train_data = 's3://lawsnic-aiml-east2/kaggle/customerChurn/features/partial/train.csv'
s3_uri_train_data_with_header = 's3://lawsnic-aiml-east2/kaggle/customerChurn/features/partial/train_with_header.csv'
s3_uri_validate_data = 's3://lawsnic-aiml-east2/kaggle/customerChurn/features/partial/validate.csv'
s3_uri_validate_data_with_target_and_header = 's3://lawsnic-aiml-east2/kaggle/customerChurn/features/partial/validate_with_target_and_header.csv'
s3_uri_validate_data_with_target = 's3://lawsnic-aiml-east2/kaggle/customerChurn/features/partial/validate_with_target.csv'

s3_input_train = sagemaker.inputs.TrainingInput(s3_data=s3_uri_train_data, content_type='csv')
s3_input_validation = sagemaker.inputs.TrainingInput(s3_data=s3_uri_validate_data_with_target, content_type='csv')

train data has 41 features
validation data has 41 features before dropping target


First we'll need to specify training parameters to the estimator.  This includes:
1. The `xgboost` algorithm container
1. The IAM role to use
1. Training instance type and count
1. S3 location for output data
1. Algorithm hyperparameters

And then a `.fit()` function which specifies:
1. S3 location for output data.  In this case we have both a training and validation set which are passed in.

In [29]:
from smexperiments.trial import Trial

trial = Trial.create(experiment_name=experiment_name, sagemaker_boto_client=sm)
experiment_config = {
    "ExperimentName": experiment_name,
    "TrialName": trial.trial_name,
    "TrialComponentDisplayName": "Data",
}

sess = sagemaker.Session()

xgb = sagemaker.estimator.Estimator(container,
                                    role, 
                                    instance_count=1, 
                                    instance_type='ml.m4.xlarge',
                                    output_path='s3://{}/{}/manual-output/xgb-hpo'.format(bucket,prefix),
                                    sagemaker_session=sess,
                                    enable_sagemaker_metrics=True)
xgb.set_hyperparameters(max_depth=5,
                        eta=0.2,
                        gamma=4,
                        min_child_weight=6,
                        subsample=0.8,
                        silent=0,
                        objective='binary:logistic',
                        eval_metric='auc',
                        num_round=100)

xgb.fit({'train': s3_input_train, 'validation': s3_input_validation},
    experiment_config=experiment_config)

INFO:sagemaker.image_uris:Defaulting to the only supported framework/algorithm version: latest.
INFO:sagemaker.image_uris:Ignoring unnecessary instance type: None.
INFO:sagemaker:Creating training-job with name: xgboost-2023-02-18-18-04-45-844


2023-02-18 18:04:46 Starting - Starting the training job...
2023-02-18 18:05:11 Starting - Preparing the instances for trainingProfilerReport-1676743485: InProgress
.........
2023-02-18 18:06:30 Downloading - Downloading input data...
2023-02-18 18:07:10 Training - Downloading the training image...
2023-02-18 18:07:42 Uploading - Uploading generated training modelArguments: train
[2023-02-18:18:07:37:INFO] Running standalone xgboost training.
[2023-02-18:18:07:37:INFO] File size need to be processed in the node: 1.03mb. Available memory size in the node: 8618.28mb
[2023-02-18:18:07:37:INFO] Determined delimiter of CSV input is ','
[18:07:37] S3DistributionType set as FullyReplicated
[18:07:37] 6337x40 matrix with 253480 entries loaded from /opt/ml/input/data/train?format=csv&label_column=0&delimiter=,
[2023-02-18:18:07:37:INFO] Determined delimiter of CSV input is ','
[18:07:37] S3DistributionType set as FullyReplicated
[18:07:37] 706x40 matrix with 28240 entries loaded from /opt/ml/in

## HPO

In [30]:
from sagemaker.tuner import IntegerParameter, CategoricalParameter, ContinuousParameter, HyperparameterTuner
hyperparameter_ranges = {'eta': ContinuousParameter(0, 1),
                            'min_child_weight': ContinuousParameter(1, 10),
                            'alpha': ContinuousParameter(0, 2),
                            'max_depth': IntegerParameter(1, 10)}

objective_metric_name = 'validation:auc'

tuner = HyperparameterTuner(xgb,
                            objective_metric_name,
                            hyperparameter_ranges,
                            max_jobs=20,
                            max_parallel_jobs=3)

In [31]:
tuner.fit({'train': s3_input_train, 'validation': s3_input_validation})

INFO:sagemaker.image_uris:Defaulting to the only supported framework/algorithm version: latest.
INFO:sagemaker.image_uris:Ignoring unnecessary instance type: None.
INFO:sagemaker:Creating hyperparameter tuning job with name: xgboost-230218-1819


...............................................................................................!


In [32]:
boto3.client('sagemaker').describe_hyper_parameter_tuning_job(
HyperParameterTuningJobName=tuner.latest_tuning_job.job_name)['HyperParameterTuningJobStatus']

'Completed'

In [33]:
boto3.client('sagemaker').describe_training_job(TrainingJobName=tuner.best_training_job())['FinalMetricDataList']

[{'MetricName': 'validation:auc',
  'Value': 0.820264995098114,
  'Timestamp': datetime.datetime(2023, 2, 18, 18, 25, 59, tzinfo=tzlocal())},
 {'MetricName': 'train:auc',
  'Value': 0.7948939800262451,
  'Timestamp': datetime.datetime(2023, 2, 18, 18, 25, 59, tzinfo=tzlocal())},
 {'MetricName': 'ObjectiveMetric',
  'Value': 0.820264995098114,
  'Timestamp': datetime.datetime(2023, 2, 18, 18, 25, 59, tzinfo=tzlocal())}]

In [38]:
#modeltuner.best_training_job()
#boto3.client('sagemaker').describe_training_job(TrainingJobName=tuner.best_training_job())

In [ ]:
#xgb_predictor = xgb.deploy(initial_instance_count=1, instance_type='ml.m4.xlarge')
xgb_predictor = tuner.deploy(initial_instance_count=1, instance_type="ml.m4.xlarge") 

In [ ]:
import sys
xgb_predictor.serializer = sagemaker.serializers.CSVSerializer()

predictionsT = xgb_predictor.predict(cleaned_test_data.to_numpy()).decode('utf-8')

#print(predictionsT)

predictions = np.fromstring(predictionsT[1:], sep='\r\n')